In [ ]:
import pandas as pd
import difflib

# ==============================
# 1. Load your global skills DB
# ==============================
df = pd.read_csv(r"D:\AI Career Consellor\database.csv")

# Clean column names (make them lowercase and strip spaces)
df.columns = df.columns.str.strip().str.lower()
print("📌 Columns in dataset:", df.columns.tolist())

# Build career -> required skills mapping
career_skill_map = df.groupby("suggested_career")["skills"].apply(list).to_dict()
career_industry_map = df.set_index("suggested_career")["major_application"].to_dict()

# ==============================
# 2. Fuzzy matching with difflib
# ==============================
def map_user_skills(user_skills):
    all_skills = df["skills"].unique().tolist()
    mapped = []
    for skill in user_skills:
        match = difflib.get_close_matches(skill.lower(), [s.lower() for s in all_skills], n=1, cutoff=0.6)
        if match:
            for s in all_skills:
                if s.lower() == match[0]:
                    mapped.append(s)
    return mapped

# ==============================
# 3. Career suggestion function
# ==============================
def suggest_career(user_skills):
    user_skills = map_user_skills(user_skills)
    results = []

    for career, required_skills in career_skill_map.items():
        required_skills_set = set(s.lower() for s in required_skills)
        user_skills_set = set(s.lower() for s in user_skills)

        matched = required_skills_set & user_skills_set
        missing = required_skills_set - user_skills_set
        match_percent = (len(matched) / len(required_skills_set) * 100) if required_skills_set else 0

        results.append({
            "career": career,
            "industry": career_industry_map.get(career, "Unknown"),
            "match_percent": round(match_percent, 1),
            "required_skills": list(required_skills_set),
            "matched_skills": list(matched),
            "missing_skills": list(missing)
        })

    # Sort careers by best match
    results = sorted(results, key=lambda x: x["match_percent"], reverse=True)
    return results[:5]  # top 5 suggestions

# ==============================
# 4. Test the system
# ==============================
if __name__ == "__main__":
    # Example user input
    user_skills = ["debating", "communication"]

    suggestions = suggest_career(user_skills)

    print(f"\n✅ Your Skills: {user_skills}\n")
    print("🎯 Top Career Suggestions:\n")
    for s in suggestions:
        print(f"{s['career']} ({s['match_percent']}% Match)")
        print(f"   📌 Industry: {s['industry']}")
        print(f"   ✅ Required: {s['required_skills']}")
        print(f"   ✨ Matched: {s['matched_skills']}")
        print(f"   ❌ Missing: {s['missing_skills']}\n")
